## Advent of Code 2021, Day 12

This might be a good example for depth-first search according to the classifier.

#### Input Processing

In [75]:
from collections import Counter, defaultdict
from copy import deepcopy
from dataclasses import dataclass
from typing import Tuple

In [30]:
example1 = """start-A
start-b
A-c
A-b
b-d
A-end
b-end
""".splitlines()

In [31]:
example2 = """dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc
""".splitlines()

In [32]:
example3 = """fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW
""".splitlines()

In [33]:
with open("inputs/2021d12") as f:
  input = [line.strip() for line in  f.readlines()]

In [34]:
Graph = dict[str, list[str]]

def parse(input: str) -> Graph:
    """
    Returns a graph represented as a hashmap.
    
    Since the graph in question is (mostly) undirected, we automatically add backedges
    (i.e., if `A` is connected to `B`, we add both `A -> [B, ...]` and `B -> [A, ...]`) entries.
    The only exceptions are `start` and `end`, since those are source and sink nodes.
    """
    graph = defaultdict(list)

    for edge in input:
        (v1, v2) = edge.split("-")

        if v1 == "start" or v2 == "end":
            graph[v1].append(v2)
        elif v1 == "end" or v2 == "start":
            graph[v2].append(v1)
        else:
          graph[v1].append(v2)
          graph[v2].append(v1)

    return graph

In [35]:
example1 = parse(example1)
example2 = parse(example2)
example3 = parse(example3)
graph = parse(input)

In [39]:
graph

defaultdict(list,
            {'LA': ['sn', 'mo', 'zs', 'end'],
             'sn': ['LA', 'mo', 'mh', 'vx', 'RD', 'JQ'],
             'mo': ['LA', 'sn', 'mh', 'JQ', 'zs', 'RD'],
             'zs': ['LA', 'end', 'JI', 'mo', 'rk', 'JQ'],
             'RD': ['end', 'mo', 'sn'],
             'start': ['vx', 'mh', 'JQ'],
             'mh': ['mo', 'sn', 'JQ', 'vx'],
             'JI': ['zs'],
             'JQ': ['mo', 'mh', 'zs', 'vx', 'sn'],
             'rk': ['zs'],
             'vx': ['sn', 'mh', 'JQ']})

### Part 1: Depth-First Search

We can then use a standard depth-first search algorithm with an added condition:

- lowercase nodes can only be visited once (uppercase can be visited any number of times).

In [76]:
@dataclass
class State:
    path: list[str]
    visited: set[str]
    visits: Counter[str]

In [73]:
def dfs(graph: Graph) -> set[Tuple[str, ...]]:
    paths = set()

    stack = [("start", [])]
    visits, visited = Counter(), set()
    while stack:
        curr, path = stack.pop()
        print((curr, path))

        if curr == "end":
            path = deepcopy(path)
            path.append(curr)
        
            paths.add(tuple(path))
            continue

        if curr not in visited:
            # TODO: can we extract it into a helper
            # can only be visited once
            if curr.islower():
                visited.add(curr)
            else:
                # uppercase nodes could be visited twice
                visits[curr] += 1
                if visits[curr] == 2:
                    visited.add(curr)

            for next in graph[curr]:
                next_path = deepcopy(path)
                next_path.append(curr)

                stack.append((next, next_path))
    
    return paths

In [74]:
dfs(example1)

('start', [])
('b', ['start'])
('end', ['start', 'b'])
('d', ['start', 'b'])
('b', ['start', 'b', 'd'])
('A', ['start', 'b'])
('end', ['start', 'b', 'A'])
('b', ['start', 'b', 'A'])
('c', ['start', 'b', 'A'])
('A', ['start', 'b', 'A', 'c'])
('end', ['start', 'b', 'A', 'c', 'A'])
('b', ['start', 'b', 'A', 'c', 'A'])
('c', ['start', 'b', 'A', 'c', 'A'])
('A', ['start'])


{('start', 'b', 'A', 'c', 'A', 'end'),
 ('start', 'b', 'A', 'end'),
 ('start', 'b', 'end')}

In [72]:
example1["start"]

['A', 'b']